In [1]:
import numpy as np
from collections import deque
from copy import deepcopy

from math import gcd

In [2]:
test0 = ['broadcaster -> a, b, c',
         '%a -> b',
         '%b -> c',
         '%c -> inv',
         '&inv -> a']

test1 = ['broadcaster -> a',
         '%a -> inv, con',
         '&inv -> b',
         '%b -> con',
         '&con -> output']

data = np.genfromtxt('day20_input.txt', dtype=str, delimiter='\n', comments=None)

In [3]:
class flipflop():
    def __init__(self, name):
        self.state = False
        self.name = name
        self.signal_name = deque()
        self.signal_value = deque()
        
    def set_network(self, network, out_connection):
        self.network = network
        self.out_connection = out_connection
        
    def signal(self, name, sig):
        self.signal_name.append(name)
        self.signal_value.append(sig)
        
    def process(self):
        if len(self.signal_name) == 0:
            return []
        
        signal_name = self.signal_name.popleft()
        signal = self.signal_value.popleft()
        
        signalled = []
        
        if signal == 0:
            if self.state:
                signal = 0
            else:
                signal = 1
            self.state = not self.state
            
            for oc in self.out_connection:
                self.network[oc].signal(self.name, signal)
                signalled.append(oc)
                #print(self.name, signal, oc)
        elif signal == 1:
            signal = -1
                
        return signalled, signal
                
class conjunction():
    def __init__(self, name):
        self.state = {}
        self.name = name
        self.signal_name = deque()
        self.signal_value = deque()
        
        self.last_signal = -1
        
    def set_input(self, in_conns):
        for ic in in_conns:
            self.state[ic] = 0
        
    def set_network(self, network, out_connection):
        self.network = network
        self.out_connection = out_connection
        
    def signal(self, name, sig):
        self.signal_name.append(name)
        self.signal_value.append(sig)
        
    def process(self):
        if len(self.signal_name) == 0:
            return []
        
        signal_name = self.signal_name.popleft()
        signal = self.signal_value.popleft()
        
        self.state[signal_name] = signal
        send_sig = 1
        if sum(self.state.values()) == len(self.state.values()):
            send_sig = 0
            
        signalled = []
        for oc in self.out_connection:
            self.network[oc].signal(self.name, send_sig)
            signalled.append(oc)
            #print(self.name, send_sig, oc)
            
        if self.name in self.network['special_nodes'] and send_sig == 1 and self.network['press'] > 0:
            if self.name not in self.network['loops'].keys():
                self.network['loops'][self.name] = self.network['press']
            
        self.last_signal = send_sig
        return signalled, send_sig
    
class broadcaster():
    def __init__(self, name):
        self.name = name
        
    def set_network(self, network, out_connection):
        self.network = network
        self.out_connection = out_connection
        
    def process(self):
        signalled = []
        for oc in self.out_connection:
            self.network[oc].signal(self.name, 0)
            signalled.append(oc)
            #print(self.name, 0, oc)
        return signalled, 0
    
class untype():
    def __init__(self, name):
        self.name = name
        self.out_connection = []
        
    def set_network(self, network, out_connection):
        return
        
    def signal(self, name, sig):
        return
    
    def process(self):
        return [], -1

In [4]:
def build_network(data):
    network = {}
    out_connection = {}
    conj = []
    for line in data:
        send, recieve = line.split(' -> ')        
        if send[0] == 'b':
            network[send] = broadcaster(send)
        elif send[0] == '%':
            send = send[1:]
            network[send] = flipflop(send)
        elif send[0] == '&':
            send = send[1:]
            network[send] = conjunction(send)
            conj.append(send)
            
        recieve = recieve.split(',')
        
        out_connection[send] = []
        for r in recieve:
            out_connection[send].append(r.strip())
        
    #set network and output nodes
    for node in network.keys():
        network[node].set_network(network, out_connection[node])
        
    #For conjunctions, we need the input nodes
    for con in conj:
        in_connection = []
        for node in network.keys():
            if node == con:
                continue
            if con in network[node].out_connection:
                in_connection.append(node)
        network[con].set_input(in_connection)
        
    #add output node(s)
    outputs = []
    for node in network.keys():
        for oc in network[node].out_connection:
            if oc not in list(network.keys()):
                outputs.append(oc)
    for oc in outputs:
        network[oc] = untype(oc)
        
    network['press'] = 0
    network['special_nodes'] = ['vg', 'kp', 'gc', 'tx']
    network['loops'] = {}
        
    return network

def push_button(network):
    sequence = deque(['broadcaster'])
    high_pulse = 0
    low_pulse = 1
    
    while len(sequence):
        node = sequence.popleft()
        signalled, pulse = network[node].process()
        
        if pulse == 0:
            low_pulse += len(signalled)
        elif pulse == 1:
            high_pulse += len(signalled)
        for s in signalled:
            sequence.append(s)
            
    return network, high_pulse, low_pulse
            
def part1(data, cycles=1000):
    network = build_network(data)
    
    high_pulse = 0
    low_pulse = 0
    
    for _ in range(0, cycles):
        network, hp, lp = push_button(network)
        high_pulse += hp
        low_pulse += lp
        
    #print(low_pulse, high_pulse)
    return high_pulse*low_pulse

print(part1(test0))
print(part1(test1))
print('Part 1 result:', part1(data))

32000000
11687500
Part 1 result: 681194780


In [5]:
def lcm(numbers):
    locomu = 1
    for i in numbers:
        locomu = locomu*i//gcd(locomu, i)
    return locomu

def part2(data):
    network = build_network(data)
    
    while True:
        network['press'] += 1
        network, _, _ = push_button(network)
            
        if len(network['loops']) == 4:
            break
            
    return lcm(list(network['loops'].values()))
        
print('Part 2 result:', part2(data))

Part 2 result: 238593356738827
